In [1]:
%load_ext autoreload
%autoreload 2

# MTBO Reizman Study

In [2]:
from summit.strategies import MTBO, Transform, LHS, Chimera
from summit.benchmarks import ReizmanSuzukiEmulator, BaumgartnerCrossCouplingEmulator
from summit.utils.dataset import DataSet
from summit.domain import *
import summit
import pathlib
import pandas as pd

## Create DataSet

In [3]:
# Path to data
summit_path = pathlib.Path(summit.__file__).parents[0]
data_path = summit_path / "benchmarks" / "experiment_emulator" / "data"

In [4]:
# Import Reizman-Suzuki Data
datasets = [DataSet.read_csv(data_path / f"reizman_suzuki_case{i}_train_test.csv") for i in range(1,5)]
for i, dataset in enumerate(datasets):
    dataset[('task', 'METADATA')] = i
datasets[0].tail(5)

,catalyst,t_res,temperature,catalyst_loading,ton,yield,task
91,P1-L6,189.7,110,1.127,6.6,7.4,0
92,P1-L4,600,110,1.052,65.4,68.7,0
93,P1-L3,600,65.3,1.106,5.1,5.6,0
94,P2-L1,600,30,0.496,1.1,0.6,0
95,P1-L2,189.7,110,2.513,21.1,52.9,0


In [5]:
exp = ReizmanSuzukiEmulator(case=1)
exp.domain

Name,Type,Description,Values
catalyst,"categorical, input",Catalyst type - different ligands,8 levels
t_res,"continuous, input",Residence time in seconds (s),"[60,600]"
temperature,"continuous, input",Reactor temperature in degrees Celsius (ºC),"[30,110]"
catalyst_loading,"continuous, input",Catalyst loading in mol%,"[0.5,2.5]"
ton,"continuous, minimize objective",Turnover number - moles product generated divided by moles catalyst used,"[0,200]"
yield,"continuous, maximize objective",Yield,"[0,100]"


In [6]:
lhs = LHS(exp.domain)
conditions = lhs.suggest_experiments(5)
exp.run_experiments(conditions)

AttributeError: 'CategoricalVariable' object has no attribute 'enc'

In [ ]:
exp.reset()
hierarchy = {"yield": {"hierarchy": 0, "tolerance": 0.5}, "ton": {"hierarchy": 1, "tolerance": 1}}
transform = Chimera(exp.domain, hierarchy=hierarchy)
strategy = MTBO(exp.domain, 
                pretraining_data=datasets[1],
                transform=transform,
                categorical_method="one-hot", 
                task=0)
r = summit.Runner(strategy=strategy, experiment=exp, max_iterations=10)
r.run()

In [ ]:
exp.pareto_plot(colorbar=True)

In [ ]:
exp.data

In [23]:
new_exp = ReizmanSuzukiEmulator(case=2)
new_exp.train(dataset=datasets[1], verbose=False)

In [25]:
lhs = LHS(new_exp.domain)
conditions = lhs.suggest_experiments(10)
new_exp.run_experiments(conditions)

,catalyst,t_res,temperature,catalyst_loading,ton,yield,computation_t,experiment_t,strategy
0,P1-L4,465.0,58.0,1.2,6.641219,14.581401,24.293291,0.060923,LHS
1,P1-L7,141.0,90.0,1.8,6.641000,14.587530,24.293291,0.065157,LHS
2,P1-L5,195.0,34.0,1.4,6.643018,14.580414,24.293291,0.062647,LHS
3,P1-L1,411.0,74.0,0.6,6.644189,14.577108,24.293291,0.056093,LHS
4,P1-L6,357.0,106.0,2.4,6.638281,14.589335,24.293291,0.055811,LHS
5,P1-L4,249.0,82.0,2.0,6.638522,14.587759,24.293291,0.055453,LHS
6,P1-L5,87.0,98.0,1.6,6.639736,14.589821,24.293291,0.056580,LHS
7,P1-L5,519.0,50.0,1.0,6.642876,14.575467,24.293291,0.056712,LHS
8,P1-L5,573.0,66.0,2.2,6.646348,14.587234,24.293291,0.057579,LHS
9,P2-L1,303.0,42.0,0.8,6.640424,14.576373,24.293291,0.058345,LHS


In [8]:
pd_cn = BaumgartnerCrossCouplingEmulator()
pd_cn.domain

Name,Type,Description,Values
catalyst,"categorical, input",Catalyst type,3 levels
base,"categorical, input",Base,4 levels
base_equivalents,"continuous, input",Base equivalents,"[1.0,2.5]"
temperature,"continuous, input",Temperature in degrees Celsius (ºC),"[30,100]"
t_res,"continuous, input",residence time in seconds (s),"[60,1800]"
yld,"continuous, maximize objective",Yield,"[0.0,1.0]"


In [16]:
lhs = LHS(pd_cn.domain, categorical_method="descriptors")
conditions = lhs.suggest_experiments(10)
pd_cn.run_experiments(conditions)

,catalyst,base,base_equivalents,temperature,t_res,yld,computation_t,experiment_t,strategy,M2,M2_cat,area,area_cat
0,tBuXPhos,TMG,1.825,61.5,669.0,0.112790,12.830038,0.032917,LHS,178.562475,694.220455,67.56765,107.424515
1,tBuXPhos,TMG,2.125,54.5,321.0,0.047158,12.830038,0.027945,LHS,191.573095,801.974065,50.86719,95.049495
2,tBuXPhos,TMG,1.375,47.5,1365.0,0.136937,12.830038,0.031420,LHS,211.089025,730.138325,28.59991,88.861985
3,tBuXPhos,TMG,2.275,40.5,495.0,0.038747,12.830038,0.027224,LHS,217.594335,514.631105,45.30037,113.612025
4,tBuXPhos,TMG,1.075,96.5,147.0,0.424852,12.830038,0.032255,LHS,224.099645,658.302585,78.70129,76.486965
5,tBuXPhos,TMG,2.425,75.5,1017.0,0.524719,12.830038,0.025872,LHS,165.551855,766.056195,62.00083,82.674475
6,tBuXPhos,TMG,1.675,82.5,843.0,0.499695,12.830038,0.026088,LHS,198.078405,622.384715,34.16673,119.799535
7,tBuXPhos,TMG,1.225,33.5,1539.0,0.100131,12.830038,0.024048,LHS,172.057165,586.466845,56.43401,70.299455
8,tBuXPhos,TMG,1.525,89.5,1191.0,0.700953,12.830038,0.025105,LHS,204.583715,550.548975,39.73355,125.987045
9,tBuXPhos,TMG,1.975,68.5,1713.0,0.582713,12.830038,0.024665,LHS,185.067785,478.713235,73.13447,101.237005
